# YOLO モデルを再トレーニングする

YOLO モデルを再トレーニングするには、中程度の事故と重大な事故のラベルが付いた準備された自動車画像のデータセットが必要です。トレーニング データセットと検証データセットに分割されたアノテーションされた画像のデータセット (RoboFlow から取得) が既にあります。これらのトレーニング/検証セットを使用して、現在の YOLO モデルを再トレーニングします。 トレーニングに必要なデータ構造に関するいくつかの情報:

1. モデルに検出を教えたいオブジェクトのエンコード クラスは、0-`moderate` と 1-`severe`.
2. データセットは独自のフォルダー内にあり、その中に 2 つのサブフォルダー `train` と `valid` があります。  各サブフォルダー内には 2 つのサブフォルダー `images` と `labels` があります。
3. 各画像には `labels` サブフォルダー内に、対応するアノテーション用テキストファイルがあります。 アノテーション用テキストファイルには、画像ファイルと同じ名前が付けられます。
4. データセット記述 YAML ファイル (data.yaml) はデータセットを示し、データセット内のオブジェクト クラスを記述します。この YAML ファイルはモデルの `train` メソッドに渡されて、トレーニング プロセスを開始します。
**Note**: テキストや音声、画像、動画などあらゆる形態のデータの1つ1つに、タグやメタデータと呼ばれる情報を付けていく工程のことをアノテーションと呼んでいます。

さぁ始めましょう！

In [ ]:
# If you did not use the Workbench image designed for this Lab, you can uncomment and run the following line to install the required packages.
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

import os
import requests
import zipfile
from tqdm.notebook import tqdm
from ultralytics import YOLO

次に、YOLO モデル 'yolo8m.pt' をロードしましょう。

In [ ]:
# Load model
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)

## トレーニングデータを取得する

次の 2 つのトレーニング データ セットが zip ファイルとして提供されています。  
1) `accident-full.zip`   - モデルを完全に再トレーニングするために使用
2) `accident-sample.zip` - モデルを完全に再トレーニングする時間がない場合に、モデルを部分的に再トレーニングするために使用

ワークショップ中はサンプル データセットのみを使用します。

In [ ]:
# Function to retrieve a specific dataset
def retrieve_dataset(dataset_type):

    # Check if the directory exists, if not, create it
    if not os.path.exists("./datasets/"):
        os.makedirs("./datasets/")

    URL = f"https://rhods-public.s3.amazonaws.com/sample-data/accident-data/accident-{dataset_type}.zip"

    # Check if the file exists, if not, download and unzip it
    if not os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Downloading file...")
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        t = tqdm(total=total_size, unit='iB', unit_scale=True)
        with open(f'./datasets/accident-{dataset_type}.zip', 'wb') as f:
            for data in response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    if os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Unzipping file...")
        with zipfile.ZipFile(f'./datasets/accident-{dataset_type}.zip', 'r') as zip_ref:
            zip_ref.extractall(path='./datasets/')
    print("Done!")


dataset_type = 'sample'
# dataset_type = 'full' # Use this line instead if you want to retrieve the full dataset
retrieve_dataset(dataset_type)

## YOLO モデルを再トレーニングする

'epoch' とは何かを理解することから始めましょう。  機械学習モデルは、アルゴリズムを通過した特定のデータセットを使用してトレーニングされます。完全なデータセットがアルゴリズムを通過するたびに、**epoch** が完了したと言われます。各 **epoch**  では、モデルのトレーニングがさらに洗練されます。

以下のトレーニング コードでは、さまざまなパラメーターを設定します。  
**results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)**

- epochs: プロセスをデモするだけなので、エポックは 1 つだけ使用します。
- imgsz: これはモデルに供給する必要がある画像のサイズです。
- batch: これは、アルゴリズムが同時に処理する画像の数です。多ければ多いほど良い結果が得られますが、消費するメモリも多くなります。このワークショップでは限られたリソースを使用しており、これは単なるトレーニング例であるため、意図的に低い値の 2 に保ちます。

トレーニングの実行では、各 **epoch<** にトレーニング フェーズと検証フェーズの両方の概要が表示されます。行 1 と 2 はトレーニングフェーズの結果を示し、行 3 と 4 は各エポックの検証フェーズの結果を示します。

次のセルを実行して、モデルの再トレーニングを開始しましょう！

In [ ]:
# Train the model

results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)

**Note**: 標準的な再トレーニングの後、結果に満足できれば、モデルを ONNX 形式にエクスポートできます。
(次のコマンドの trainX を使用するトレーニングセッションに置き換えます。)

`ObjDetOXModel = YOLO("runs/detect/trainX/weights/best.pt").export(format="onnx")`

## トレーニング結果の解釈

**full dataset** に対するトレーニングのプロセスと結果の詳細な説明は、ラボの手順で参照できます。

モデルを再トレーニングしたので、自動車事故のある画像に対してテストしてみましょう！

**ノートブック `04-04-accident-recog.ipynb` を開いてください** 。